In [11]:
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import itertools 
import numpy as np
import plotly.figure_factory as ff
import scipy
# importing Statistics module
import statistics
  
#df = pd.read_csv('07_09_21_18_13_record.csv')
#df = pd.read_csv('07_19_21_22_26_record.csv')
df = pd.read_csv('07_29_21_16_37_record.csv', skiprows= 11)
df_P40 = pd.read_csv('P20SV3020210819A02(Black Stem)11-9-2021.csv', skiprows=11)
df_P1KS = pd.read_csv('P1KSV3020210819A03(w-probe_attached)-11-9-2021.csv', skiprows=11)
df_P1KS_anodized_removed = pd.read_csv('P1KSV3020210819A03(removed Anodized).csv', skiprows=11)
df_P1KS_anodized = pd.read_csv('P1KSV3020210819A03(Anodized-probe_attached).csv', skiprows = 11)


In [12]:
# make a list of trial strings to use later
def calculate_trials(n):
    trials = []
    for t in range(1, n+1):
        trials.append('trial {}'.format(t))
    return trials

def obtain_dataframe(dataframe, trials):
    new_df = dataframe[
            [
             'Dial Indicator(mm)',
             'Phase']
           ]

    triggered_distance = []
    for trial in trials:
        p_data = new_df.loc[new_df['Phase'] == trial]
        point = p_data['Dial Indicator(mm)'][p_data.index[-1]]
        triggered_distance.append(point)

    accuracy = pd.DataFrame({'Triggered Point(mm)': triggered_distance, 'Trials': [i for i in range(1,len(trials)+1)]})
    return accuracy


In [15]:

#------------------------------------
trials = calculate_trials(500)
accuracy = obtain_dataframe(df_P40, trials)


#---------------------------------------

trials_2 = calculate_trials(152)
accuracy_2 = obtain_dataframe(df, trials_2)

#------------------------------------------------


trials_3 = calculate_trials(408)
accuracy_3 = obtain_dataframe(df_P1KS, trials_3)

#--------------------------------------------------

trials_3 = calculate_trials(408)
accuracy_3 = obtain_dataframe(df_P1KS, trials_3)

#--------------------------------------------------

x  = list(accuracy['Triggered Point(mm)'])
y = list(accuracy_2['Triggered Point(mm)'])
z = list(accuracy_3['Triggered Point(mm)'])
fig = ff.create_distplot([accuracy['Triggered Point(mm)'], accuracy_2['Triggered Point(mm)'], accuracy_3['Triggered Point(mm)']], 
                         ['(P40S-w/o Probe)', 'P3HS-Proto-3-w/ Probe', 'P1KS-w/ Probe'], 
                         bin_size=0.01, 
                         #histnorm = 'probability',
                         curve_type='normal')

fig.update_xaxes(title_text = 'Triggered Position(mm)')
fig.update_yaxes(title_text = 'Frequency(%)')
fig.update_layout(title_text='Capacitive Triggered Displacement',
                         #xaxis = dict(dtick=4),
                         yaxis = dict(dtick=2))
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='show')
fig.show()


In [16]:
# use subplots for secondary y axis
single_graph = make_subplots(specs=[[{"secondary_y": False}]])
trials = calculate_trials(1)
new_df = df_P1KS_anodized[
            [
             'Time',
             'Capacitance',
             'Dial Indicator(mm)',
             'Phase']
           ]

cp_df = df_P1KS_anodized_removed[
                            [
                             'Time',
                             'Capacitance',
                             'Dial Indicator(mm)',
                             'Phase']
                            ]

for trial in trials:
    if trial == 'trial 1':
        p_data = new_df.loc[new_df['Phase']  == trial]
        single_graph.add_trace(go.Scatter(x=p_data['Time'], 
                                 y=p_data['Capacitance'],
                                 mode='lines+markers',
                                 name='P1KS(Anodized)-{}'.format(trial)),
                                 secondary_y=False)

        p_data_2 = cp_df.loc[cp_df['Phase'] == trial]
        single_graph.add_trace(go.Scatter(x=p_data_2['Time'], 
                                 y=p_data_2['Capacitance'],
                                 mode='lines+markers',
                                 name='P1KS(Removed Anodized)-{}'.format(trial)),
                                  secondary_y=False)
        
        single_graph.update_xaxes(title_text = 'Time(mins)')
        single_graph.update_yaxes(title_text = 'Capacitance(pF)', secondary_y = False)
        single_graph.update_layout(title='Capacitance Test',
                           xaxis = dict(dtick= 0.5),
                           yaxis = dict(dtick=0.5)
                           )
    else:
        #print(trial)
        p_data = new_df.loc[new_df['Phase']  == trial]
        time_dataframe = [0]
        for x in range(1, len(p_data['Time'])):
            new_time = p_data.values[x][0]-p_data.values[0][0]
            time_dataframe.append(new_time)
        #print(time_dataframe)
        time_frame = pd.DataFrame({'Time': time_dataframe})

        single_graph.add_trace(go.Scatter(x=p_data['Time'], 
                                 y=p_data['Capacitance'],
                                 mode='lines+markers',
                                 name='Capacitance-{}'.format(trial)),
                                  secondary_y=False)

        single_graph.update_xaxes(title_text = 'Time(mins)')
        single_graph.update_yaxes(title_text = 'Capacitance with Dial Indicator', secondary_y = False)
        single_graph.update_layout(title='Capacitance',
                           xaxis = dict(dtick=4),
                           yaxis = dict(dtick=1)
                           )

iplot(single_graph)

In [17]:
single_graph = make_subplots(specs=[[{"secondary_y": False}]])
trials = calculate_trials(500)


new_df = df_P40[
            [
             'Time',
             'Dial Indicator(mm)',
             'Trial',
             'Phase']
           ]
p_data = obtain_dataframe(new_df, trials)
single_graph.add_trace(go.Scatter(x=p_data['Trials'], 
                                 y=p_data['Triggered Point(mm)'],
                                 mode='lines+markers',
                                 name='Capacitance-{}'.format(trial)),
                                  secondary_y=False)
        
single_graph.update_xaxes(title_text = 'Trials')
single_graph.update_yaxes(title_text = 'Triggered Point(mm)', secondary_y = False)
single_graph.update_layout(title='Capacitance Test',
                   xaxis = dict(dtick= 10),
                   yaxis = dict(dtick=0.05)
                    )

iplot(single_graph)